## Library System

In [1]:
## import necessary lib
import psycopg2
import json
import pandas as pd
import datetime
import math
from urllib.request import urlopen

/Users/peiqichen/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
with open('db_cred.json','r') as j:
    cred = json.load(j)
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(
    host=cred["host"],
    port=cred['port'],
    database=cred["database"],
    user=cred["user"],
    password=cred["password"])
# create a cursor
cur = conn.cursor()
# execute a statement
print('PostgreSQL database version:')
cur.execute('SELECT version()')

# display the PostgreSQL database server version
db_version = cur.fetchone()
print(db_version)

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 14.8 (Homebrew) on aarch64-apple-darwin22.4.0, compiled by Apple clang version 14.0.3 (clang-1403.0.22.14.1), 64-bit',)


In [3]:
df = pd.read_csv("books.csv",on_bad_lines='skip')
print(df.shape[0])

11127


In [4]:
from pymongo import MongoClient
client = MongoClient('localhost',27017) ## or MongoClient("localhost:27")
db = client.bigdata

In [5]:
conn.rollback()
delCmd = """ DROP TABLE sources """
cur.execute(delCmd)
delCmd = """ DROP TABLE timestamps """
cur.execute(delCmd)
delCmd = """ DROP TABLE serials """
cur.execute(delCmd)
delCmd = """ DROP TABLE authorInfo """
cur.execute(delCmd)

In [6]:
createCmd = """ CREATE TABLE serials (
                bookID CHAR(5) PRIMARY KEY,
                isbn VARCHAR(32) NOT NULL,
                isbn13 VARCHAR(32) NOT NULL)
            """
cur.execute(createCmd)
createCmd = """ CREATE TABLE sources (
                bookID CHAR(5),
                authors VARCHAR(1024) NOT NULL,
                language_code VARCHAR(8) NOT NULL,
                average_rating INT,
                ratings_count INT,
                publisher VARCHAR(1024),
                primary key (bookID),
foreign key (bookID) references serials
                )
             """
cur.execute(createCmd)
createCmd = """ CREATE TABLE timestamps (
                bookID CHAR(5),
                title VARCHAR(256) NOT NULL,
                num_pages VARCHAR(16) NOT NULL,
                text_reviews_count VARCHAR(16),
                publication_date DATE,
                primary key (bookID),
foreign key (bookID) references serials
                )
            """

cur.execute(createCmd)
conn.commit()

### Insert value

In [7]:
# conn.rollback()
for i in range(0, df.shape[0]-1):  
    bookID = str(df.iloc[i]["bookID"]) 
    authors = "'"+ df.iloc[i]["authors"].replace("\'","\''") +"'"
    title = "'" + df.iloc[i]["title"].replace("\'","\''")+"'"
    isbn = "'" + str(df.iloc[i]["isbn"])+ "'"
    if(df.iloc[i]["isbn13"].find("X") >= 0): 
        isbn13 = str(df.iloc[i]["isbn13"])
    else:
        isbn13 = str(math.trunc(float(df.iloc[i]["isbn13"])))
    ratings_count = str(df.iloc[i]["ratings_count"]) 
    publisher = "'"+ df.iloc[i]["publisher"].replace("\'","\''")  + "'"
    text_reviews_count = str(df.iloc[i]["text_reviews_count"])
    average_rating = df.iloc[i]["average_rating"]
    num_pages = df.iloc[i]["  num_pages"]
    if(df.iloc[i]["publication_date"].find("/") < 0): continue
    try:
        publication_date =  "'"+    datetime.datetime.strptime(df.iloc[i]["publication_date"], "%m/%d/%Y").strftime("%Y-%m-%d") + "'"
    except ValueError:
        print(ValueError)
        continue
    language_code = "'"+ df.iloc[i]["language_code"] + "'"
    
    
    cmd = "INSERT into serials VALUES(" + bookID + ","  + isbn + "," + isbn13 +")"
    print(cmd)
    cur.execute(cmd)
    cmd = "INSERT into sources VALUES(" + bookID + ","+ authors + "," + language_code + "," + average_rating + "," + ratings_count + "," + publisher+")"
    print(cmd)
    cur.execute(cmd)
    cmd = "INSERT into timestamps VALUES("+ bookID + "," + title + "," + num_pages + "," + text_reviews_count + "," + publication_date+")"
    print(cmd)
    cur.execute(cmd)
    
conn.commit()

INSERT into serials VALUES(1,'439785960',9780440000000)
INSERT into sources VALUES(1,'J.K. Rowling/Mary GrandPré','eng',4.57,2095690,'Scholastic Inc.')
INSERT into timestamps VALUES(1,'Harry Potter and the Half-Blood Prince (Harry Potter  #6)',652,27591,'2006-09-16')
INSERT into serials VALUES(2,'439358078',9780440000000)
INSERT into sources VALUES(2,'J.K. Rowling/Mary GrandPré','eng',4.49,2153167,'Scholastic Inc.')
INSERT into timestamps VALUES(2,'Harry Potter and the Order of the Phoenix (Harry Potter  #5)',870,29221,'2004-09-01')
INSERT into serials VALUES(4,'439554896',9780440000000)
INSERT into sources VALUES(4,'J.K. Rowling','eng',4.42,6333,'Scholastic')
INSERT into timestamps VALUES(4,'Harry Potter and the Chamber of Secrets (Harry Potter  #2)',352,244,'2003-11-01')
INSERT into serials VALUES(5,'043965548X',9780440000000)
INSERT into sources VALUES(5,'J.K. Rowling/Mary GrandPré','eng',4.56,2339585,'Scholastic Inc.')
INSERT into timestamps VALUES(5,'Harry Potter and the Prisoner 

INSERT into sources VALUES(290,'Kahlil Gibran/جبران خليل جبران','eng',3.98,1308,'Knopf')
INSERT into timestamps VALUES(290,'Jesus the Son of Man',216,82,'1995-02-21')
INSERT into serials VALUES(291,'806501901',9780810000000)
INSERT into sources VALUES(291,'Kahlil Gibran/جبران خليل جبران/Anthony Rizcallah Ferris','eng',3.92,6432,'Citadel')
INSERT into timestamps VALUES(291,'The Broken Wings',132,517,'2003-03-03')
INSERT into serials VALUES(292,'067943920X',9780680000000)
INSERT into sources VALUES(292,'Kahlil Gibran/جبران خليل جبران','eng',4.08,3015,'Knopf')
INSERT into timestamps VALUES(292,'Sand and Foam',100,217,'2011-06-14')
INSERT into serials VALUES(295,'753453800',9780750000000)
INSERT into sources VALUES(295,'Robert Louis Stevenson','eng',3.83,318753,'Kingfisher')
INSERT into timestamps VALUES(295,'Treasure Island',311,6734,'2001-09-15')
INSERT into serials VALUES(296,'1402714572',9781400000000)
INSERT into sources VALUES(296,'Robert Louis Stevenson/Scott McKowen','en-US',3.83,4

INSERT into serials VALUES(822,'60733357',9780060000000)
INSERT into sources VALUES(822,'Neal Stephenson','en-US',4.26,19320,'William Morrow Paperbacks')
INSERT into timestamps VALUES(822,'The Confusion (The Baroque Cycle  #2)',815,565,'2005-06-14')
INSERT into serials VALUES(823,'60593083',9780060000000)
INSERT into sources VALUES(823,'Neal Stephenson','eng',3.93,30872,'HarperCollins Perennial')
INSERT into timestamps VALUES(823,'Quicksilver (The Baroque Cycle  #1)',927,1735,'2004-09-21')
INSERT into serials VALUES(824,'553383442',9780550000000)
INSERT into sources VALUES(824,'Neal Stephenson/J. Frederick George','eng',3.61,2465,'Spectra Books')
INSERT into timestamps VALUES(824,'The Cobweb',448,137,'2005-05-31')
INSERT into serials VALUES(826,'380816032',9780380000000)
INSERT into sources VALUES(826,'Neal Stephenson','en-US',3.25,4657,'William Morrow Paperbacks')
INSERT into timestamps VALUES(826,'The Big U',308,223,'2001-02-06')
INSERT into serials VALUES(827,'553380966',97805500000

INSERT into timestamps VALUES(1366,'The Histories',653,17,'1970-08-30')
INSERT into serials VALUES(1367,'393959465',9780390000000)
INSERT into sources VALUES(1367,'Herodotus/Jennifer Tolbert Roberts/Walter Blanco','en-US',3.99,40,'W.W. Norton & Company')
INSERT into timestamps VALUES(1367,'The Histories',464,5,'1992-01-17')
INSERT into serials VALUES(1368,'375400613',9780380000000)
INSERT into sources VALUES(1368,'Herodotus/Edward Henry Blakeney/George Rawlinson/Rosalind Thomas','en-US',3.99,132,'Everyman''s Library 234')
INSERT into timestamps VALUES(1368,'The Histories',816,10,'1997-03-25')
INSERT into serials VALUES(1369,'226327701',9780230000000)
INSERT into sources VALUES(1369,'Herodotus/Peter Grene','en-US',3.99,36,'University of Chicago Press')
INSERT into timestamps VALUES(1369,'The History',710,6,'1987-03-14')
INSERT into serials VALUES(1371,'140275363',9780140000000)
INSERT into sources VALUES(1371,'Homer/Robert Fagles/Bernard Knox','eng',3.86,288792,'Penguin Classics')
INSER

INSERT into sources VALUES(1658,'Karen  O''Connor/Larry J. Sabato','eng',2.83,0,'Longman Publishing Group')
INSERT into timestamps VALUES(1658,'American Government: Continuity and Change  Alternate Edition',664,0,'2005-03-11')
INSERT into serials VALUES(1664,'321365208',9780320000000)
INSERT into sources VALUES(1664,'Karen  O''Connor/Larry J. Sabato','eng',3.5,0,'Longman Publishing Group')
INSERT into timestamps VALUES(1664,'Essentials of American and Texas Government: Continuity and Change',854,0,'2005-07-29')
INSERT into serials VALUES(1667,'8432216429',9788430000000)
INSERT into sources VALUES(1667,'Ernesto Sabato','spa',4.05,453,'Seix Barral')
INSERT into timestamps VALUES(1667,'El túnel',159,60,'2003-07-01')
INSERT into serials VALUES(1681,'1565480848',9781570000000)
INSERT into sources VALUES(1681,'Augustine of Hippo/John E. Rotelle/Maria Boulding','eng',3.92,142,'New City Press')
INSERT into timestamps VALUES(1681,'The Confessions (Works of Saint Augustine 1)',416,24,'2002-12-01

...

INSERT into sources VALUES(43089,'Eric Hobsbawm/Chris Wrigley','eng',3.92,301,'The New Press')
INSERT into timestamps VALUES(43089,'Industry and Empire: The Birth of the Industrial Revolution',411,14,'1999-09-01')
INSERT into serials VALUES(43098,'1859848982',9781860000000)
INSERT into sources VALUES(43098,'Karl Marx/Friedrich Engels/Eric Hobsbawm','eng',3.55,309,'Verso')
INSERT into timestamps VALUES(43098,'The Communist Manifesto',88,38,'1998-05-01')
INSERT into serials VALUES(43101,'1565846710',9781570000000)
INSERT into sources VALUES(43101,'Eric Hobsbawm/Allan Cameron/Antonio Polito','eng',3.71,82,'New Press')
INSERT into timestamps VALUES(43101,'On the Edge of the New Century',192,5,'2001-05-01')
INSERT into serials VALUES(43106,'349106711',9780350000000)
INSERT into sources VALUES(43106,'Eric Hobsbawm','eng',4.25,348,'Abacus')
INSERT into timestamps VALUES(43106,'The Age of Extremes: The Short Twentieth Century  1914-1991',638,24,'1995-11-12')
INSERT into serials VALUES(43110,'8

INSERT into serials VALUES(43691,'743250419',9780740000000)
INSERT into sources VALUES(43691,'Colm Tóibín','eng',3.83,7299,'Scribner')
INSERT into timestamps VALUES(43691,'The Master',339,771,'2005-05-03')
INSERT into serials VALUES(43700,'140149864',9780140000000)
INSERT into sources VALUES(43700,'Colm Tóibín','eng',3.6,807,'Penguin Books')
INSERT into timestamps VALUES(43700,'The South',240,92,'1992-10-01')
INSERT into serials VALUES(43702,'743203313',9780740000000)
INSERT into sources VALUES(43702,'Colm Tóibín','eng',3.87,4421,'Scribner')
INSERT into timestamps VALUES(43702,'The Blackwater Lightship',288,367,'2005-06-05')
INSERT into serials VALUES(43704,'330373579',9780330000000)
INSERT into sources VALUES(43704,'Colm Tóibín','eng',3.8,23,'Picador USA')
INSERT into timestamps VALUES(43704,'The Sign of the Cross: Travels in Catholic Europe',243,7,'2001-05-04')
INSERT into serials VALUES(43706,'091827382X',9780920000000)
INSERT into sources VALUES(43706,'Karen Tei Yamashita','eng',3.

INSERT into serials VALUES(44309,'375700234',9780380000000)
INSERT into sources VALUES(44309,'Seamus Deane','en-US',3.73,3237,'Vintage')
INSERT into timestamps VALUES(44309,'Reading in the Dark',256,227,'1998-02-24')
INSERT into serials VALUES(44312,'61122181',9780060000000)
INSERT into sources VALUES(44312,'Elizabeth Noble','en-US',3.59,5796,'William Morrow Paperbacks')
INSERT into timestamps VALUES(44312,'Alphabet Weekends',448,386,'2007-01-23')
INSERT into serials VALUES(44321,'1877058335',9781880000000)
INSERT into sources VALUES(44321,'Babette Smith','eng',3.58,0,'Rosenberg Publishing')
INSERT into timestamps VALUES(44321,'A Cargo of Women: Susannah Watson and the Convicts of the Princess Royal',264,0,'2005-06-01')
INSERT into serials VALUES(44323,'64407454',9780060000000)
INSERT into sources VALUES(44323,'Francesca Lia Block','en-US',3.77,6872,'Joanna Cotler Books/HarperCollinsPublishers')
INSERT into timestamps VALUES(44323,'The Rose and the Beast: Fairy Tales Retold',240,395,'2

INSERT into serials VALUES(44988,'812971345',9780810000000)
INSERT into sources VALUES(44988,'Rudyard Kipling/Pankaj Mishra','eng',3.72,113,'Modern Library')
INSERT into timestamps VALUES(44988,'Kim',306,22,'2004-02-10')
INSERT into serials VALUES(45000,'1400078679',9781400000000)
INSERT into sources VALUES(45000,'Russell Shorto','eng',4.14,6919,'Vintage')
INSERT into timestamps VALUES(45000,'The Island at the Center of the World: The Epic Story of Dutch Manhattan and the Forgotten Colony That Shaped America',384,777,'2005-04-12')
INSERT into serials VALUES(45017,'761545646',9780760000000)
INSERT into sources VALUES(45017,'Craig Keller','eng',4.17,6,'Prima Games')
INSERT into timestamps VALUES(45017,'The Bard''s Tale: Prima''s Official Strategy Guide',144,0,'2004-10-26')
INSERT into serials VALUES(45018,'067187778X',9780670000000)
INSERT into sources VALUES(45018,'Mercedes Lackey','eng',4.18,1530,'Baen')
INSERT into timestamps VALUES(45018,'The Free Bards (Bardic Voices  #1-3)',712,20,

INSERT into serials VALUES(45574,'1932394613',9781930000000)
INSERT into sources VALUES(45574,'Dave Crane/Eric Pascarello/Darren James','en-US',3.56,126,'Manning Publications')
INSERT into timestamps VALUES(45574,'Ajax in Action',680,10,'2005-11-03')
INSERT into serials VALUES(45583,'8466318771',9788470000000)
INSERT into sources VALUES(45583,'Rosa Montero','spa',3.9,1266,'Punto de Lectura')
INSERT into timestamps VALUES(45583,'Historia del rey transparente',592,90,'2006-09-01')
INSERT into serials VALUES(45585,'843221728X',9788430000000)
INSERT into sources VALUES(45585,'Rosa Montero','spa',3.66,293,'Planeta Publishing')
INSERT into timestamps VALUES(45585,'Bella y oscura',204,31,'2006-03-01')
INSERT into serials VALUES(45592,'8466302298',9788470000000)
INSERT into sources VALUES(45592,'Mario Vargas Llosa','spa',3.92,162,'Suma')
INSERT into timestamps VALUES(45592,'La tía Julia y el escribidor',566,10,'2001-03-01')
INSERT into serials VALUES(45595,'8432203238',9788430000000)
INSERT in

### PostgreSQL Queries

In [8]:
createCmd = """ CREATE TABLE authorInfo (
                author VARCHAR(1024) NOT NULL,
                bio VARCHAR(2048) NOT NULL,
                works VARCHAR(2048) NOT NULL,
                primary key (author)
                )
            """

cur.execute(createCmd)
conn.commit()

In [9]:
name_olid = {"J.K. Rowling": "OL23919A",
             "Stephen King": "OL2162284A",
             "Gabriel García Márquez": "OL4586796A"
            }
name_works = {}
name_bio = {}

for k in name_olid:
    # grab bio and amazon links
    url = "https://openlibrary.org/authors/" + name_olid[k] + ".json"
    response = urlopen(url)
    data_json = json.loads(response.read())
    bio = data_json['bio']
    source_records = data_json['source_records']
    name_bio[k] = bio
    name_works[k] = []
    ww = ""
    # add amazon link of works
    for src in source_records:
#         link = 'https://www.amazon.com/s?k=' + src.replace('amazon:','')
        link = src.replace('amazon:','')
        name_works[k].append(link)
        ww += link+"&"
    # update database
    if type(bio) is dict:
        bio = bio["value"].replace("\"","")
    elif type('') is str:
        bio = bio.replace("\"","")
#     print(bio)
    cmd = "INSERT into authorInfo VALUES(\'"+ k + "\',\'" + bio + "\',\'" + ww +"\')"
    print(cmd)
    cur.execute(cmd)
    
    print("documents updated.")

INSERT into authorInfo VALUES('J.K. Rowling','Joanne Jo Murray, OBE (née Rowling), better known under the pen name J. K. Rowling, is a British author best known as the creator of the Harry Potter fantasy series, the idea for which was conceived whilst on a train trip from Manchester to London in 1990. The Potter books have gained worldwide attention, won multiple awards, sold more than 400 million copies, and been the basis for a popular series of films.','8893817055&4863895666&1526618249&8862561512&1526618230&6073193009&0747574286&8327158775&841810791X&2246810663&3548285287&7020150705&bwb:9788498382686&9123513683&1526618354&6041160008&1526622807&5353044592&060641519X&8327150731&bwb:9780751579956&9750843878&9124083836&286866086X&2075094551&3551559201&9124087181&1804220272&bwb:9781526624536&0606323473&8983927623&8418174404&0747598460&8498389119&8498388260&8532531776&886715818X&849838267X&bwb:9788418797125&9124087173&0606415181&ia:harrypotteresfon0000rowl&4915512495&0807281751&bwb:978607

### Flask

In [10]:
from flask import Flask, render_template
from flask import session, url_for

In [ ]:
author_list = [
    ['J.K. Rowling','/J.K. Rowling'],
    ['Stephen King','/Stephen King'],
    ['Gabriel García Márquez','/Gabriel García Márquez']
]
app = Flask("flask_demo")

@app.route('/')
def index():
    return render_template('index.html', result=author_list)

@app.route('/<name>')
def author(name):
    print(name_works[name])
    cmd = "SELECT bio FROM authorInfo WHERE author=\'"+name +"\'"
    cur.execute(cmd)
    row = cur.fetchone()
    bio = ""
    while row is not None :
        bio += row[0]
        row = cur.fetchone() 
    print(bio)
    cmd = "SELECT works FROM authorInfo WHERE author=\'"+name +"\'"
    cur.execute(cmd)
    row = cur.fetchone()
    works = ""
    while row is not None :
        works += row[0]
        row = cur.fetchone() 
    works = works.split("&")[:-1]
    print(works)
    return render_template('info.html', name=name, bio = bio, works = works)
#     return f'You are in author page {name}'

    
# with app.test_request_context():
#     print(url_for('index'))
#     print(url_for('profile', username='J.K. Rowling'))
app.run(host='localhost', port=5001)

 * Serving Flask app 'flask_demo' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5001 (Press CTRL+C to quit)
127.0.0.1 - - [09/Aug/2023 18:15:05] "GET /J.K.%20Rowling HTTP/1.1" 200 -
[2023-08-09 18:15:05,354] ERROR in app: Exception on /favicon.ico [GET]
Traceback (most recent call last):
  File "/Users/peiqichen/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/peiqichen/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1519, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/peiqichen/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1517, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/peiqichen/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1503, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/var/folders/91/6_yhrz3d0lx4sn8d90p006qm0000gp/T/ipykernel_68483/28646124.py", line 14, in author
    print(name_work

['8893817055', '4863895666', '1526618249', '8862561512', '1526618230', '6073193009', '0747574286', '8327158775', '841810791X', '2246810663', '3548285287', '7020150705', 'bwb:9788498382686', '9123513683', '1526618354', '6041160008', '1526622807', '5353044592', '060641519X', '8327150731', 'bwb:9780751579956', '9750843878', '9124083836', '286866086X', '2075094551', '3551559201', '9124087181', '1804220272', 'bwb:9781526624536', '0606323473', '8983927623', '8418174404', '0747598460', '8498389119', '8498388260', '8532531776', '886715818X', '849838267X', 'bwb:9788418797125', '9124087173', '0606415181', 'ia:harrypotteresfon0000rowl', '4915512495', '0807281751', 'bwb:9786073816779', 'bwb:9781484474112', '1551926431', 'ia:haeripotopulsajo0000rowl_g0v1', '6555320222', '9124083895', '912408719X', '9124078867', 'ia:harrypotterylapi0000rowl_q5r6', '8498384451', '2840116464', 'bwb:9780316263825', '0545850568', 'bwb:9798769805622', '1551927004', 'promise:bwb_daily_pallets_2021-06-17', '1547904135', '0

127.0.0.1 - - [09/Aug/2023 18:15:06] "GET /J.K.%20Rowling HTTP/1.1" 200 -
[2023-08-09 18:15:06,136] ERROR in app: Exception on /favicon.ico [GET]
Traceback (most recent call last):
  File "/Users/peiqichen/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/peiqichen/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1519, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/peiqichen/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1517, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/peiqichen/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1503, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/var/folders/91/6_yhrz3d0lx4sn8d90p006qm0000gp/T/ipykernel_68483/28646124.py", line 14, in author
    print(name_works[name])
KeyError: 'favicon.ico'
127.0.0.1 - - [09/Aug/2023

['8893817055', '4863895666', '1526618249', '8862561512', '1526618230', '6073193009', '0747574286', '8327158775', '841810791X', '2246810663', '3548285287', '7020150705', 'bwb:9788498382686', '9123513683', '1526618354', '6041160008', '1526622807', '5353044592', '060641519X', '8327150731', 'bwb:9780751579956', '9750843878', '9124083836', '286866086X', '2075094551', '3551559201', '9124087181', '1804220272', 'bwb:9781526624536', '0606323473', '8983927623', '8418174404', '0747598460', '8498389119', '8498388260', '8532531776', '886715818X', '849838267X', 'bwb:9788418797125', '9124087173', '0606415181', 'ia:harrypotteresfon0000rowl', '4915512495', '0807281751', 'bwb:9786073816779', 'bwb:9781484474112', '1551926431', 'ia:haeripotopulsajo0000rowl_g0v1', '6555320222', '9124083895', '912408719X', '9124078867', 'ia:harrypotterylapi0000rowl_q5r6', '8498384451', '2840116464', 'bwb:9780316263825', '0545850568', 'bwb:9798769805622', '1551927004', 'promise:bwb_daily_pallets_2021-06-17', '1547904135', '0

127.0.0.1 - - [09/Aug/2023 18:15:09] "GET / HTTP/1.1" 200 -
[2023-08-09 18:15:09,348] ERROR in app: Exception on /favicon.ico [GET]
Traceback (most recent call last):
  File "/Users/peiqichen/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/peiqichen/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1519, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/peiqichen/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1517, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/peiqichen/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1503, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/var/folders/91/6_yhrz3d0lx4sn8d90p006qm0000gp/T/ipykernel_68483/28646124.py", line 14, in author
    print(name_works[name])
KeyError: 'favicon.ico'
127.0.0.1 - - [09/Aug/2023 18:15:09] "GE

['8893817055', '4863895666', '1526618249', '8862561512', '1526618230', '6073193009', '0747574286', '8327158775', '841810791X', '2246810663', '3548285287', '7020150705', 'bwb:9788498382686', '9123513683', '1526618354', '6041160008', '1526622807', '5353044592', '060641519X', '8327150731', 'bwb:9780751579956', '9750843878', '9124083836', '286866086X', '2075094551', '3551559201', '9124087181', '1804220272', 'bwb:9781526624536', '0606323473', '8983927623', '8418174404', '0747598460', '8498389119', '8498388260', '8532531776', '886715818X', '849838267X', 'bwb:9788418797125', '9124087173', '0606415181', 'ia:harrypotteresfon0000rowl', '4915512495', '0807281751', 'bwb:9786073816779', 'bwb:9781484474112', '1551926431', 'ia:haeripotopulsajo0000rowl_g0v1', '6555320222', '9124083895', '912408719X', '9124078867', 'ia:harrypotterylapi0000rowl_q5r6', '8498384451', '2840116464', 'bwb:9780316263825', '0545850568', 'bwb:9798769805622', '1551927004', 'promise:bwb_daily_pallets_2021-06-17', '1547904135', '0